## PExpect
Spawn a child application and control it as if a human were typing commands.

There are two main interfaces to the Pexpect system; these are the function, `run()` and the class, `spawn`.

### ```run```

* The run() function is simpler than spawn

* good for quickly calling program. 

* run() function it executes a given program and then returns the output. 

* This is a handy replacement for os.system().


In [2]:
import pexpect
pexpect.run('ls -la')

'total 72\r\ndrwxrwxrwx  6 abdulmuneer  staff    204 Feb 26 16:26 \x1b\x1b.\x1b[m\x1b[m\r\ndrwxrwxrwx  7 abdulmuneer  staff    238 Feb 21 08:34 \x1b\x1b..\x1b[m\x1b[m\r\n-rw-r--r--@ 1 abdulmuneer  staff   6148 Feb 21 08:34 .DS_Store\r\ndrwxr-xr-x  4 abdulmuneer  staff    136 Feb 26 16:14 \x1b.ipynb_checkpoints\x1b[m\x1b[m\r\n-rw-r--r--  1 abdulmuneer  staff   2529 Feb 26 16:26 pexpect,fork, Pipes, timers, signal handling, Environment variables.ipynb\r\n-rwxrwxrwx  1 abdulmuneer  staff  20793 Feb 25 19:22 \x1bthreads_multiprocess_subprocess.ipynb\x1b[m\x1b[m\r\n'

### ```spawn```

* more powerful interface to the Pexpect system. 
* use this to spawn a child program then interact with it
* sending input and expecting responses

two important methods in Pexpect – 
#### - expect() 
#### - send() 
   (or sendline() which is like send() with a linefeed). 
 
 The expect() method waits for the child application to return a given string. 
 The string you specify is a regular expression, so you can match complicated patterns. 
 
 The send() method writes a string to the child application. From the child’s point of view it looks just like someone typed the text from a terminal. 

After each call to expect() the ```before``` and ```after``` properties will be set to the text printed by child application. 

The before property will contain all text up to the expected string pattern. 

The after string will contain the text that was matched by the expected pattern. The match property is set to the re match object.

```python
import pexpect
child = pexpect.spawn ('ftp ftp.openbsd.org')
child.expect ('Name .*: ')
child.sendline ('anonymous')
child.expect ('Password:')
child.sendline ('noah@example.com')
child.expect ('ftp> ')
child.sendline ('ls /pub/OpenBSD/')
child.expect ('ftp> ')
print child.before   # Print the result of the ls command.
child.interact()     # Give control of the child to the user.
```

#### Timeout
```python
child.expect('password:', timeout=120)
```

#### End of line:
```python
child.expect('\r\n')
```

Pexpect uses a Pseudo-TTY device to talk to the child application, 
so when the child app prints "\n" you actually see "\r\n".

Simple "\n" is not sufficient

#### careful with
```python
child.expect ('.*') 
child.expect ('.+')
```

### Expecting a list of possible outcomes

```python
child.expect('password:')
child.sendline(my_secret_password)
# We expect any of these three patterns...
i = child.expect (['Permission denied', 'Terminal type', '[#\$] '])
if i==0:
    print('Permission denied on host. Can\'t login')
    child.kill(0)
elif i==1:
    print('Login OK... need to send terminal type.')
    child.sendline('vt100')
    child.expect('[#\$] ')
elif i==2:
    print('Login OK.')
    print('Shell command prompt', child.after)
    
```


Pexpect does NOT interpret shell meta characters such as redirect, pipe, or wild cards ( >, |, or * )

If you *do* want to include those, you need to start a shell.
e.g:
```python
child = pexpect.spawn('/bin/bash -c "ls -l | grep LOG > logs.txt"')
child.expect(pexpect.EOF)
```

## Pipes

pipes – Unix shell command pipeline

 Inputs and outputs of the commands can be chained together like the  | operator

In [5]:
! echo 'hello world' | tr a-z A-Z

HELLO WORLD


In [33]:
import pipes
pipe_template = pipes.Template()
pipe_template.append('tr a-z A-Z', '--')
tr = pipe_template.open('pipe_upper_output', 'w')
tr.write('hello world')
tr.close()

In [37]:
print open('pipe_upper_output').read()

HELLO WORLD


In [45]:
#using file for input

import pipes
import tempfile

p = pipes.Template()
p.debug(True)
p.append('grep -n HELLO $IN', 'f-')

t = tempfile.NamedTemporaryFile('r')

p.copy('pipe_upper_output', t.name)

t.seek(0)
print t.read()
t.close()

IN=pipe_upper_output; grep -n HELLO $IN >/var/folders/ty/bxzhn_yj77nffxvv4xf95kkc0000gn/T/tmpH8sQgH
1:HELLO WORLD



## Fork

In [47]:
"forks child processes until you type 'q'"
import os
def child():
    print('Hello from child', os.getpid())
    os._exit(0) # else goes back to parent loop

def parent():
    while True:
        newpid = os.fork() 
        #os.fork() returns 0 inside the child process (this is standard for Unix systems) 
        # and the child PID inside the parent.
        
        if newpid == 0:
            child()
        else:
            print('Hello from parent', os.getpid(), newpid)
        if raw_input() == 'q': break

parent()

('Hello from parent', 950, 1243)
d
('Hello from parent', 950, 1244)
('Hello from child', 1243)
e
('Hello from parent', 950, 1245)
('Hello from child', 1244)
d
('Hello from parent', 950, 1247)
('Hello from child', 1245)
q
('Hello from child', 1247)


generally speaking, using os.fork() is a bad idea and needs to be done carefully at best.

#### Any open files, sockets etc, get duplicated by os.fork(). This is not what a library expects. 

##### So in general,

* Don't fork while any sockets or database connections are open
* Don't fork while any files are open
* Don't fork in a multithreaded process, because this makes it almost impossible to guarantee the above
* Don't fork after you've initialised any nontrivial library that might internally use sockets, temp files, threads etc.

And finally:

* If you did fork, don't exit the process normally (including with an unhandled exception!). 
* Instead use either os._exit, or os.execl. 
* This ensures that the child process won't "clean up" resources that the parent is still using.

The problem is, even assuming that all of those are true when you first write your code, using os.fork() puts these requirements (possibly invisibly) on all future maintenance-programmers.

## Scheduling

* implements a generic event scheduler for running tasks at specific times. 
* The scheduler class uses a time function to learn the current time, 
* and a delay function to wait for a specific period of time. 

In [48]:
import sched
import time

scheduler = sched.scheduler(time.time, time.sleep)

def print_event(name):
    print 'EVENT:', time.time(), name

print 'START:', time.time()
scheduler.enter(2, 1, print_event, ('first',))
scheduler.enter(3, 1, print_event, ('second',))

scheduler.run()

START: 1456500500.4
EVENT: 1456500502.4 first
EVENT: 1456500503.4 second


## Signals

Used to receive notification of asynchronous system events

They can be generated by the system itself, or sent from one process to another. Since signals interrupt the regular flow of your program, it is possible that some operations (especially I/O) may produce error if a signal is received in the middle.

Signals are identified by integers and are defined in the operating system C headers. Python exposes the signals appropriate for the platform as symbols in the signal module.



#### receiving signals:

In [ ]:
import signal
import os
import time

def receive_signal(signum, stack):
    print 'Received:', signum

signal.signal(signal.SIGUSR1, receive_signal)
signal.signal(signal.SIGUSR2, receive_signal)

print 'My PID is:', os.getpid()

while True:
    print 'Waiting...'
    time.sleep(3)
    
#kill -USR1 $pid, kill -USR2 $pid, and kill -INT $pid

#### sending signals
```python
os.kill(pid, signal.SIGUSR1)
```

#### Alarms
Alarms are a special sort of signal, where your program asks the OS to notify it after some period of time has elapsed.



In [57]:
import signal
import time

def receive_alarm(signum, stack):
    print 'Alarm :', time.ctime()

# Call receive_alarm in 2 seconds
signal.signal(signal.SIGALRM, receive_alarm)
signal.alarm(2)

print 'Before:', time.ctime()
time.sleep(5)
print 'After :', time.ctime()

Before: Fri Feb 26 21:30:30 2016
Alarm : Fri Feb 26 21:30:32 2016
After : Fri Feb 26 21:30:32 2016


#### getsignal()
To see what signal handlers are registered for a signal, use getsignal(). 

* Pass the signal number as argument. 
* The return value is the registered handler, 
* or one of the special values:

  - signal.SIG_IGN (if the signal is being ignored), 
  - signal.SIG_DFL (if the default behavior is being used), 
  - None (if the existing signal handler was registered from C, rather than Python).

In [59]:
import signal

def alarm_received(n, stack):
    return

signal.signal(signal.SIGALRM, alarm_received)

signals_to_names = {}
for n in dir(signal):
    if n.startswith('SIG') and not n.startswith('SIG_'):
        signals_to_names[getattr(signal, n)] = n

for s, name in sorted(signals_to_names.items()):
    handler = signal.getsignal(s)
    if handler is signal.SIG_DFL:
        handler = 'SIG_DFL'
    elif handler is signal.SIG_IGN:
        handler = 'SIG_IGN'
    print '%-10s (%2d):' % (name, s), handler

SIGHUP     ( 1): SIG_IGN
SIGINT     ( 2): <built-in function default_int_handler>
SIGQUIT    ( 3): SIG_DFL
SIGILL     ( 4): SIG_DFL
SIGTRAP    ( 5): SIG_DFL
SIGIOT     ( 6): SIG_DFL
SIGEMT     ( 7): SIG_DFL
SIGFPE     ( 8): SIG_DFL
SIGKILL    ( 9): None
SIGBUS     (10): SIG_DFL
SIGSEGV    (11): SIG_DFL
SIGSYS     (12): SIG_DFL
SIGPIPE    (13): SIG_IGN
SIGALRM    (14): <function alarm_received at 0x103f27b18>
SIGTERM    (15): SIG_DFL
SIGURG     (16): SIG_DFL
SIGSTOP    (17): None
SIGTSTP    (18): SIG_DFL
SIGCONT    (19): SIG_DFL
SIGCHLD    (20): SIG_DFL
SIGTTIN    (21): SIG_DFL
SIGTTOU    (22): SIG_DFL
SIGIO      (23): SIG_DFL
SIGXCPU    (24): SIG_DFL
SIGXFSZ    (25): SIG_IGN
SIGVTALRM  (26): SIG_DFL
SIGPROF    (27): SIG_DFL
SIGWINCH   (28): SIG_DFL
SIGINFO    (29): SIG_DFL
SIGUSR1    (30): <function receive_signal at 0x103ee4de8>
SIGUSR2    (31): <function receive_signal at 0x103ee4de8>


## Environment variables

In [ ]:
import os
os.environ


In [ ]:
os.environ.get('PATH')
